In [29]:
pwd

'/Users/veerpalsingh/Documents/Medical chatbot-RAG/Medical-Chatbot/research'

In [30]:
import os 
os.chdir("../")

In [31]:
pwd

'/Users/veerpalsingh/Documents/Medical chatbot-RAG/Medical-Chatbot'

In [36]:
# from langchain.document_loaders import PyPDFLoader, DirectoryLoader 
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [37]:
def load_pdfs(data):
    loader=DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents=loader.load()
    return documents
        

In [33]:
conda list


# packages in environment at /opt/anaconda3/envs/medicalbot:
#
# Name                          Version          Build            Channel
aiohappyeyeballs                2.6.1            pypi_0           pypi
aiohttp                         3.13.2           pypi_0           pypi
aiohttp-retry                   2.9.1            pypi_0           pypi
aiosignal                       1.4.0            pypi_0           pypi
annotated-types                 0.7.0            pypi_0           pypi
anyio                           4.11.0           pypi_0           pypi
appnope                         0.1.4            py311hca03da5_0
argon2-cffi                     21.3.0           pyhd3eb1b0_0
argon2-cffi-bindings            25.1.0           py311h254cc4a_0
asttokens                       3.0.0            py311hca03da5_0
async-lru                       2.0.5            py311hca03da5_0
attrs                           25.4.0           py311hca03da5_2
babel                           2.16.0           p

In [38]:
extracted_data=load_pdfs("data")
# extracted_data

In [32]:
from typing import List
from langchain.schema import Document
def filter_to_minimal_docs(extracted_data):
    minimal_docs:List[Document] =[]
    for doc in extracted_data:
        src=doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source":src}
            )
            )
    return minimal_docs
                
        

In [33]:
minimal_docs=filter_to_minimal_docs(extracted_data)

In [75]:
# minimal_docs

In [35]:
def text_split(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    texts_chunk=text_splitter.split_documents(minimal_docs)
    return texts_chunk
    

In [36]:
texts_chunk=text_split(minimal_docs)

In [39]:
len(texts_chunk)

5961

In [41]:
# texts_chunk

In [41]:
from langchain_community.embeddings import HuggingFaceEmbeddings
def download_embedding():
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embeddings= HuggingFaceEmbeddings(
    model_name=model_name
    )
    return embeddings
embedding=download_embedding()

In [42]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [48]:
vectors=embedding.embed_query("My name is veerpal singh ")
len(vectors)

384

In [43]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [44]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

In [45]:
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

In [46]:
from pinecone import Pinecone
pinecone_api_key=PINECONE_API_KEY
pc=Pinecone(api_key=pinecone_api_key)
pc

In [47]:
from pinecone import ServerlessSpec
index_name="medical-chatbot"
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region ="us-east-1")
    
    )
index=pc.Index(index_name)

In [48]:
from langchain_pinecone import PineconeVectorStore
doc_search=PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

NameError: name 'texts_chunk' is not defined

In [51]:
from langchain_pinecone import PineconeVectorStore
doc_search=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [52]:
retriever=doc_search.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [53]:
retrieved_docs=retriever.invoke("what is acne?")
retrieved_docs

[Document(id='8193502e-3a79-4d42-b0cf-03ea172cecb7', metadata={'source': 'data/Medical_book.pdf'}, page_content='Nancy J. Nordenson\nAcid reflux see Heartburn\nAcidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'),
 Document(id='fe60076a-1ae5-42b1-b918-1fa895176958', metadata={'source': 'data/Medical_book.pdf'}, page_content='used to clear up mild to moderately severe acne.\nIsotretinoin (Accutane) is prescribed only for very\nsevere, disfiguring acne.\nAcne is a skin condition that occurs when pores or\nhair follicles become blocked. This allows a waxy\nmaterial, sebum, to collect 

In [54]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [55]:
system_prompt=(
    "You are a medical assistant for question-answering task."
    "Use the following retrieved context to answer "
    "the question. If you don't know the answer , say that you "
    "don't know . Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


In [56]:
prompt= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [57]:
llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1,
    max_retries=2,
    timeout=None,
    max_tokens=None
)

In [67]:
rag_chain=(
    {"context": lambda x: retriever.invoke(x["input"]), 
     "input": lambda x: x["input"]
    }
    | prompt
    | llm
)

In [80]:
response=rag_chain.invoke({"input" : "In which human personality Fibroid tumors generall occurs and is their percentage ?"})


In [81]:
response

AIMessage(content='Fibroid tumors generally occur in 30-40% of women over age 40. The provided context does not specify any particular human personality type associated with their occurrence.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--f7ce8868-b63a-4544-9388-4dbc69a6c10c-0', usage_metadata={'input_tokens': 625, 'output_tokens': 218, 'total_tokens': 843, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 181}})